In [ ]:
import os
from matplotlib import pyplot as plt
import numpy as np
import sagemaker
from sagemaker import get_execution_role
from sagemaker.tensorflow import TensorFlow

sagemaker_session = sagemaker.Session()

role = get_execution_role()
region = sagemaker_session.boto_session.region_name

### Datos

In [ ]:
training_data_uri = "s3://images-itam-denoising/raw/train/"

### Entrenamiento

- Puedes usar un GPU `ml.p3.2xlarge`, pero como queremos mantenernos en el Free Tier usamos `ml.m5.xlarge`.

In [ ]:
encoder_estimator = TensorFlow(
    entry_point="gaussian-denoising-model.py",
    role=role,
    instance_count=2,
    instance_type="ml.m5.xlarge",
    framework_version="2.1.0",
    py_version="py3",
    distribution={"parameter_server": {"enabled": True}},
)

In [ ]:
# Construir y compilar el modelo del autoencoder
autoencoder = build_autoencoder()
autoencoder.summary()

Debemos configurar bien lo que sigue:

In [ ]:
# Config del problema de optimizacion
encoder_estimator.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
# entrenamiento
encoder_estimator.fit(noisy_images, original_images, batch_size=16, epochs=250, verbose=1, validation_split=0.1)